# First clean

Removing duplicate albums based on 1) album name; 2) album id; 3) feature difference

In [3]:
setwd("/home/pasoneto/Documents/github/doc_suomi/code")
source("utils.R")
source("data_cook.R")

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



In [2]:
new = fread("sample2.csv")
old = fread("grande_sem_low.csv")

base = dplyr::bind_rows(new, old)

In [3]:
replicates = base %>%
    group_by(album_id) %>%
    summarise(overall = sum(valence, energy, loudness, tempo), name = unique(name))

replicates = replicates[order(-replicates$overall), ]

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



In [4]:
#Calculate 0 differences
valores = c(replicates$overall)
album_ids = c(replicates$album_id)
nome = c(replicates$name)

difs = c()
albums = c()
nomes = c()
nomes2 = c()
for(j in 1:(length(valores)-1)){
    difs[[j]] = abs(valores[j] - valores[j+1])
    albums[[j]] = album_ids[j]
    nomes[[j]] = nome[j]
    nomes2[[j]] = nome[j+1]
    }

replicates = tibble(album_id = albums, difference =difs, nome1 = nomes, nomes2 = nomes2)

In [5]:
nomes_iguais = replicates[order(replicates$difference, replicates$nome1),] %>% filter(nome1 == nomes2)
nomes_iguais = c(nomes_iguais$album_id)

In [6]:
dif_zero = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference == 0)
dif_zero = c(dif_zero$album_id)

In [7]:
dif_pequena = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference <= 0.005)
dif_pequena = c(dif_pequena$album_id)

In [8]:
filtro = unique(c(nomes_iguais, dif_zero, dif_pequena))

In [9]:
base %>%
    filter(!album_id %in% filtro) %>% 
    dplyr::select(album_id, track_number, name, valence, energy, loudness, tempo, danceability) %>%
    write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data.csv")

# Second clean

Removing albums with wrong number of tracks (from 1 to 10, but missing track 9)

In [24]:
setwd("/home/pasoneto/Documents/github/doc_suomi/data/treated_data")

base = fread("data.csv")
#If album length != largest track_number, remove album 
raw = base %>% group_by(album_id, track_number) %>%
        summarise(valence = mean(valence), energy = mean(energy), 
                  loudness = mean(loudness), tempo = mean(tempo)) %>%
        mutate(album_length = NROW(track_number)) %>% ungroup() %>%
        group_by(album_id) %>%
        filter(max(track_number) != unique(album_length))

albums_out = c(unique(raw$album_id))

base %<>% 
    filter(!album_id %in% albums_out) %>%
    write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data.csv")

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



character(0)